<a href="https://colab.research.google.com/github/jasonpark9001/NLP/blob/main/Building_a_Semantic_Search_Engine_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Semantic Search Engine to Search for Queries with Transformers

# Semantic Search
Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines, that only finds documents based on lexical matches, semantic search can also find synonyms.


## Background
The idea behind semantic search is to embedd all entries in your corpus, which can be sentences, paragraphs, or documents, into a vector space. 

At search time, the query is embedded into the same vector space and the closest embedding from your corpus are found. These entries should have a high semantic overlap with the query.

![SemanticSearch](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png) 


## Similarity Computation

For small corpora (up to about 100k entries) we can compute the cosine-similarity between the query and all entries in the corpus.

For small corpora with few example sentences we compute the embeddings for the corpus as well as for our query.

We then use the [util.pytorch_cos_sim()](../../../docs/usage/semantic_textual_similarity.md) function to compute the cosine similarity between the query and all corpus entries.

For large corpora, sorting all scores would take too much time. Hence, we can use [torch.topk](https://pytorch.org/docs/stable/generated/torch.topk.html) to only get the top k entries.

[Reference](https://github.com/UKPLab/sentence-transformers/tree/master/examples/applications/semantic-search)


## Objective

For today's objective we will create a corpus of around 50000 question titles asked on Quora from an open dataset. Your task will be to compute sentence embeddings and then try to retrieve top 5 similar questions from the corpus for a few example queries mentioned below.

Use [Sentence Transformers](https://github.com/UKPLab/sentence-transformers) which provides a scalable way to generate document embeddings using transformers



## Load Dependencies

In [ ]:
!pip install transformers

In [ ]:
!pip install -U sentence-transformers

In [3]:
import transformers

In [4]:
import pandas as pd
import numpy as np

## Download and Load Corpus of Questions

In [ ]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

In [6]:
df = pd.read_csv('quora_duplicate_questions.tsv', sep='\t').head(25000)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
corpus = df['question1'].tolist() + df['question2'].tolist()

In [9]:
len(corpus)

50000

## Use Sentence Transformers and Generate Corpus Embeddings

Reference:

[Semantic Search Tutorial](https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/semantic-search/semantic_search.py)


# 1. Load Pre-trained Embedder Model

Load the __`roberta-large-nli-stsb-mean-tokens`__ model to generate embeddings

In [10]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = 'roberta-large-nli-stsb-mean-tokens'
embedder = SentenceTransformer(model)

#  2. Generate Corpus Embeddings

Generate embeddings for each and every document using the pre-trained model

In [12]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [13]:
corpus_embeddings.shape

torch.Size([50000, 1024])

# 3. Create a function to print top K similar sentences for a given query

Use cosine similarity by leveraging the pytorch utility in `sentence_transformers` as depicted in the previously linked tutorial.

In [15]:
from sentence_transformers import util
import torch

def print_similar_sentences(queries, model_embedder, corpus_embeddings, top_k):
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k = top_k)
        
        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")
        
        for score, idx in zip(top_results[0], top_results[1]):
            print(corpus[idx], "(score: {:.4f})".format(score))

    

# 4. Perform Semantic Search on Sample Questions to get Similar Queries from the Corpus

In [17]:
s = 'What is the step by step guide to invest'
print_similar_sentences(queries=[s],
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is the step by step guide to invest

Top 5 most similar sentences in corpus:
What is the step by step guide to invest in share market? (score: 0.8431)
What are the best investment strategy for beginners? (score: 0.7725)
What are the ways to get an investment for startup? (score: 0.7692)
How do I invest in stock market? (score: 0.7558)
How much money will I need to start investing in stock market? (score: 0.7447)


In [18]:
s = 'What is Data Science?'
print_similar_sentences(queries=[s],
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is Data Science?

Top 5 most similar sentences in corpus:
What is data science (score: 0.9840)
What is actually a data science? (score: 0.9609)
What does a data scientist do? (score: 0.8919)
What is big data science? (score: 0.8633)
What is the difference between data science and data analysis? (score: 0.7723)


In [19]:
s = 'What is natural language processing?'
print_similar_sentences(queries=[s],
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: What is natural language processing?

Top 5 most similar sentences in corpus:
How does natural language processing work? (score: 0.9242)
Which are the best schools for studying natural language processing? (score: 0.6843)
What is the english word for "अंत्योदय"? (score: 0.6685)
What are natural numbers? (score: 0.6590)
Who owns Natural Factors? (score: 0.6589)


In [20]:
s = ['Best Harry Potter Movie?','What is the best smartphone?','What is the best starter pokemon?','Batman or Superman?']

print_similar_sentences(queries=s,
                        model_embedder=embedder, 
                        corpus_embeddings=corpus_embeddings,
                        top_k=5)





Query: Best Harry Potter Movie?

Top 5 most similar sentences in corpus:
Which Harry Potter movie is the best? (score: 0.9560)
Which is the best Harry Potter movie? (score: 0.9456)
Which is your favourite Harry Potter movie and why? (score: 0.8769)
Where were the Harry Potter movies shot? (score: 0.8664)
Where was Harry Potter filmed? (score: 0.8336)




Query: What is the best smartphone?

Top 5 most similar sentences in corpus:
What are the best smartphones? (score: 0.9829)
What are the best smartphones? (score: 0.9829)
What is the best smartphone to date? (score: 0.9759)
What are the best Smartphones tech gadgets? (score: 0.9262)
Which is the best smartphone to buy now? (score: 0.9253)




Query: What is the best starter pokemon?

Top 5 most similar sentences in corpus:
How do you choose the right starter pokemon in any game? (score: 0.8680)
What is the best Pokemon GO hack? (score: 0.7935)
Which set of starter Pokemon would you choose considering all generations and why? (score